In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


In [ ]:
# Skapa en tom DataFrame
data = pd.DataFrame()
num_rows = 1000  # Antal rader i datasetet
# Fylla DataFrame med slumpmässig värden
data['Country'] = np.random.choice(['Country_A', 'Country_B', 'Country_C'], size=num_rows)
data['Political Instability'] = np.random.randint(1, 11, size=num_rows)
data['Economic Situation'] = np.random.randint(1, 11, size=num_rows)
data['Conflict Level'] = np.random.randint(1, 11, size=num_rows)
data['Environmental Disasters'] = np.random.randint(1, 11, size=num_rows)
data['Previous Refugees'] = np.random.randint(100, 10000, size=num_rows)
data['Demographic Changes'] = np.random.randint(1, 11, size=num_rows)
data['Humanitarian Aid'] = np.random.randint(1, 11, size=num_rows)
data['International Relations'] = np.random.randint(1, 11, size=num_rows)

# Visa de första raderna i datasetet
print(data.head())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Definiera funktioner 
X = data.drop('Previous Refugees', axis=1)  
y = data['Previous Refugees']  
X_encoded = pd.get_dummies(X, columns=['Country'])

# Dela upp datasetet i tränings- och testuppsättningar (80% träningsdata, 20% testdata)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skapa en ColumnTransformer för att tillämpa One-Hot Encoding på den kategoriska variabeln 'Country'
column_transformer = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), ['Country'])],
    remainder='passthrough'
)

# Använd ColumnTransformer för att transformera både tränings- och testdata
X_train_encoded = column_transformer.fit_transform(X_train)
X_test_encoded = column_transformer.transform(X_test)

# Skapa och träna en linjär regressionsmodell
model_encoded = LinearRegression()
model_encoded.fit(X_train_encoded, y_train)

# Gör förutsägelser på testdatan med vår modell
predictions = model_encoded.predict(X_test_encoded)

# Förutsäg antalet flyktingar för testdata
predictions_encoded = model_encoded.predict(X_test_encoded)

# Utvärdera modellen med hjälp av Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error (MAE):", mae)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Utvärdera modellen med RMSE och R^2
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
print("Root Mean Squared Error (RMSE):", rmse)
print("R^2 Score:", r2)

# Optimering av modellen
# använding GridSearchCV för att söka efter de bästa hyperparametrarna för modellen
params = {'fit_intercept': [True, False]}
grid_search = GridSearchCV(LinearRegression(), params, cv=5)
grid_search.fit(X_train_encoded, y_train)

# Visa de bästa parametrarna
print("Bästa parametrar:", grid_search.best_params_)

# Träna en ny modell med de bästa parametrarna
best_model = grid_search.best_estimator_
best_model.fit(X_train_encoded, y_train)

# Gör nya förutsägelser och utvärdera modellen
new_predictions = best_model.predict(X_test_encoded)
new_mae = mean_absolute_error(y_test, new_predictions)
new_rmse = np.sqrt(mean_squared_error(y_test, new_predictions))
new_r2 = r2_score(y_test, new_predictions)

print("Mean Absolute Error (MAE) för den optimerade modellen:", new_mae)
print("Root Mean Squared Error (RMSE) för den optimerade modellen:", new_rmse)
print("R^2 Score för den optimerade modellen:", new_r2)


In [ ]:
import matplotlib.pyplot as plt

# Resultaten diagraam
labels = ['MAE', 'RMSE', 'R^2']
original_scores = [mae, rmse, r2]
optimized_scores = [new_mae, new_rmse, new_r2]

# Skapa stapeldiagrammet
x = list(range(len(labels)))  # Konvertera range till lista
width = 0.45

fig, ax = plt.subplots()
rects1 = ax.bar([val - width/2 for val in x], original_scores, width, label='Ursprunglig modell')
rects2 = ax.bar([val + width/2 for val in x], optimized_scores, width, label='Optimerad modell')

# Lägg till titel och etiketter på axlarna
ax.set_ylabel('Scores')
ax.set_title('Model Metrics')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Annotera staplarna med deras värden
def autolabel(rects):
    """Anpassa etiketter för stapeldiagram."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 poäng vertikalt förskjutna
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.show()

In [ ]:
# nya data som representerar förhållandena i ett land eller en region
new_data = pd.DataFrame({
    'Country': ['Country_A'],
    'Political Instability': [7],
    'Economic Situation': [5],
    'Conflict Level': [8],
    'Environmental Disasters': [3],
    'Demographic Changes': [6],
    'Humanitarian Aid': [4],
    'International Relations': [7]
})
new_data_encoded = column_transformer.transform(new_data)
# Använd  tränade modell för att göra en förutsägelse baserat på de nya datan
predicted_refugees = best_model.predict(new_data_encoded)

# Visa mängden flyktingar
predicted_refugees_int = int(predicted_refugees[0])# indexera och konvertera till int
print("Förutsagt antal flyktingar:", predicted_refugees_int)
